In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

c:\Users\Maxim\Desktop\hw\X5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model='intfloat/multilingual-e5-large'

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2, device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained(base_model, device_map='cuda')

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
peft_model_id = 'model f1=0.718475073313783, r=32, lora_alpha=64, lora_dropout=0.1, learning_rate=0.0001/'
model = PeftModel.from_pretrained(model, peft_model_id)

In [5]:
dataset = load_from_disk('splitted_dataset/')

In [ ]:
results = []
for i in tqdm(dataset['train'].to_list()):
    inputs = tokenizer(i['text'], truncation=True, padding=True, return_tensors="pt").to('cuda')
    with torch.no_grad():
        outputs = model(**inputs).logits.argmax().item()
        results.append(outputs)

 57%|███████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                              | 159/279 [00:03<00:02, 43.14it/s]

In [ ]:
test_results = []
for i in tqdm(dataset['test'].to_list()):
    inputs = tokenizer(i['text'], truncation=True, padding=True, return_tensors="pt").to('cuda')
    with torch.no_grad():
        outputs = model(**inputs).logits.argmax().item()
        test_results.append(outputs)

In [ ]:
def compute_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
compute_metrics(dataset['train']['label'], results)

In [ ]:
compute_metrics(dataset['test']['label'], test_results)